In [15]:
import torch
import torchvision.models as models
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import torch.nn as nn
import torch.optim as optim

# Check for GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device, ' used')
# Load SqueezeNet
squeezenet = models.squeezenet1_1(pretrained=True).to(device)

cuda  used


In [ ]:
# Modify the output layer for binary classification
    #access the 2nd layer of the model
in_features = squeezenet.classifier[1].in_channels

    #kernal_size = larger number increase to learn from larger patterns at the cost of computational demand 
    #stride = larger number reduces number of pixels sampled reducing computational demand 
squeezenet.classifier[1] = torch.nn.Conv2d(in_features, 2, kernel_size=(1, 1), stride=(1, 1))

# Image format requirments. Models tend to be trained on image sizes of 224. It is important to maintain aspect ration. 
    #transforms.ToTensor formats the image to be compatable with the model. 
transform = transforms.Compose([transforms.Resize((224, )),
                                transforms.ToTensor()])

# Retriving the data source and applying transofmation. 

#data source type 1
##dataset = datasets.ImageFolder(root='path/to/mosquito_dataset', transform=transform)

# Randomly splits the dataset into training and validation sets. 80% train and 20% validation is a common ratio. 
##train_size = int(0.8 * len(dataset))
##val_size = len(dataset) - train_size
##train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


#data source type 2

train_dataset = datasets.ImageFolder(root=
                                     'D:\Programming stuff\Machine learning\Training_data\Data\Training',transform=transform)
val_dataset = datasets.ImageFolder(root=
                                     'D:\Programming stuff\Machine learning\Training_data\Data\Validation',transform=transform)

print('data loaded')


# Defining data loader settings to optimize processing.  
batch_size = 32

    #shuffling training data is important for preventing the model from learning the order. 
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    #shuffling is not needed on validation data as a consisten order makes it easier to track performance. 
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


# Loss Function - quantifies how well the model is performing by calculating difference between predicted output and true labels.
    #for a binary model 'torch.nn.CrossEntropyLoss' is commenly used.     
criterion = nn.CrossEntropyLoss()

# Optimizer - responsible for adjusting the model's parameters during the training process. 
    #lr'learning rate' controls the the size of changes made in the model.  
    #Adam - adaptive moment estimation. Well-suited for many tasks. Maintains adaptive learning rates.
optimizer = optim.Adam(squeezenet.parameters(), lr=0.001)
    
    #SGD - Updates parameters in the opposite direction of gradient. Require careful tuning of the learning rate.
#optimizer = optim.SGD(squeezenet.parameters(), lr=0.001, momentum=0.9)


print('training loop started')
#Training loop 
num_epochs = 10
for epoch in range(num_epochs):
    squeezenet = squeezenet.cuda()
    squeezenet.train()

    for inputs, labels in train_loader:
        optimizer.zero_grad() #removes gradiant from previous epoch
        inputs, labels = inputs.cuda(), labels.cuda()  # Move data to GPU
        outputs = squeezenet(inputs)
        loss = criterion(outputs, labels)
        loss.backward() #calculates gradiants 
        optimizer.step() #adjust paramaters based on gradiants 

    # Validation
    squeezenet.eval()
    with torch.no_grad(): #Disables gradiant calculations
        total_correct = 0
        total_samples = 0

        for inputs, labels in val_loader:
            inputs, labels = inputs.cuda(), labels.cuda()  # Move data to GPU
            outputs = squeezenet(inputs)
            _, predicted = torch.max(outputs, 1)
            total_samples += labels.size(0)
            total_correct += (predicted == labels).sum().item()

        accuracy = total_correct / total_samples
        print(f'Epoch [{epoch+1}/{num_epochs}], Validation Accuracy: {accuracy:.4f}')
print('complete')